# Processamento e análise dos dados


In [1]:
# Preparando ambiente (importando bibliotecas e downloads...)

!pip install nltk
import nltk
nltk.download('rslp')
nltk.download('stopwords')
nltk.download('punkt')
import re
import pandas as pd 

[nltk_data] Downloading package rslp to /home/karine/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package stopwords to /home/karine/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/karine/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Importando dataset
df = pd.read_csv("dataset.csv", sep="," , encoding="utf8")
#df = df[:500]

### Removendo e tratando dados

In [3]:
# Remove columns e create column
df.drop(columns=['id', 'text_en', 'sentiment'], axis=1, inplace=True)

# Texto para minusculo
text_lower = [t.lower() for t in df['text_pt']]
df['text_pt'] = text_lower

### Calculando um Score para frases com o SentiLex

In [4]:
sentilexpt = open('SentiLex-lem-PT02.txt','r', encoding='utf8')

In [5]:
#ontolp = open('lexico_v3.0.txt','r', encoding='utf8')

In [6]:
#subjectivity = open('Priberam_Subjectivity_Lexicon_PT.txt','r', encoding='utf8')

In [7]:
dic_palavra_polaridade = {}

In [8]:
for i in sentilexpt:
    pos_ponto = i.find('.')
    palavra = (i[:pos_ponto])
    pol_pos = i.find('POL')
    polaridade = (i[pol_pos+7:pol_pos+9]).replace(';','')
    dic_palavra_polaridade[palavra] = polaridade
#for i in ontolp:
#    split_dic = i.split(',')
#    palavra = split_dic[0]
#    polaridade = split_dic[2]
#    dic_palavra_polaridade[palavra] = polaridade

In [9]:
def Score_sentimento(frase):
    frase = frase.lower()
    l_sentimento = []
    for p in frase.split():
        l_sentimento.append(int(dic_palavra_polaridade.get(p, 0)))
    score = sum(l_sentimento)
    if score > 0:
        return 1
    elif score < 0:
        return -1
    else:
        return 0

In [10]:
df['classification'] = df['text_pt'].apply(lambda x: Score_sentimento(x))

### Stemmer e stopwords

In [11]:
# funcao para remover brackets
def remove_brackets(column):
    for x in range(1,len(column)):
        return(re.sub('[\[\]]','',repr(column)))

In [12]:
from nltk.tokenize import word_tokenize 
stop_words = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()

# Trabalhar com stemmer e stopwords da base de treinamento/teste

for x in range(0,len(df['text_pt'])):

    # Remover as stop words do texto
    word_tokens = word_tokenize(df['text_pt'][x]) 
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    
    # Remover sufixos 
    line=[]
    text_tokenized = word_tokenize((remove_brackets(filtered_sentence)))
    line =  [stemmer.stem(word) for word in text_tokenized]
    df['text_pt'][x] = (remove_brackets(line))

/home/karine/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [13]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer

# Regex para remover alguns valores do dataset  (simbolos, numeros...)
token = RegexpTokenizer(r'[a-zA-Z0-9]+')

# Cria o 'vetorizador' de acordo com os parametros abaixo
cv = CountVectorizer(lowercase=True,stop_words=None,ngram_range = (1,2),tokenizer = token.tokenize)

# Matrixsparse da representação da coluna  text_pt
text_counts= cv.fit_transform(df['text_pt'])

In [14]:
# Vocabulario
cv.vocabulary_

{'vez': 308750,
 'sr': 278912,
 'costn': 71481,
 'arrum': 25103,
 'film': 125977,
 'temp': 288240,
 'necess': 200250,
 'al': 12018,
 'm': 178051,
 'terr': 290346,
 'seq': 269240,
 'resgat': 252742,
 'qual': 241866,
 'pouc': 232584,
 'simples': 273219,
 'import': 152783,
 'nenhum': 200944,
 'person': 223854,
 'mai': 180302,
 'fantasm': 120888,
 'arm': 24284,
 'cost': 71424,
 'real': 247678,
 'log': 175286,
 'in': 153681,
 'ci': 55796,
 'esquec': 111203,
 't': 284266,
 'dev': 86336,
 'arrog': 25011,
 'superconfi': 282528,
 'ashton': 25520,
 'kutch': 167800,
 'problem': 237384,
 's': 260702,
 'garot': 136699,
 'pens': 220812,
 'melh': 185971,
 'qualqu': 242142,
 'outr': 211856,
 'pesso': 225554,
 'red': 250073,
 'mostr': 194487,
 'sin': 273554,
 'desorden': 84707,
 'nic': 202442,
 'obst': 206959,
 'cul': 74405,
 'parec': 216286,
 'est': 111594,
 'venc': 306905,
 'fin': 129025,
 'b': 32087,
 'mei': 185639,
 'c': 43983,
 'cont': 68012,
 'sobr': 275241,
 'inform': 156695,
 'lev': 171636,
 'p

# Testando o modelo 

In [15]:
# Importando biliotecas para selecao de amostra, modelo e avaliação do modelo.
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

# Divindo no dataset em treino e teste
X_train, X_test, y_train, y_test = train_test_split(text_counts, df['classification'], 
                                                    test_size=0.34, random_state=1, 
                                                    shuffle=True)
# Criar modelo e treinar
clf = MultinomialNB().fit(X_train, y_train)

# Fazendo  predict do valor de X para teste de acuracidade
y_predicted= clf.predict(X_test)
print("MultinomialNB Accuracy:",metrics.accuracy_score(y_test, y_predicted).round(3))

MultinomialNB Accuracy: 0.581


In [16]:
from leia import SentimentIntensityAnalyzer 

s = SentimentIntensityAnalyzer()

# Análise de texto simples
s.polarity_scores('Eu estou feliz')
#{'neg': 0.0, 'neu': 0.328, 'pos': 0.672, 'compound': 0.6249}

# Análise de texto com emoji :)
s.polarity_scores('Eu estou feliz :)')
#{'neg': 0.0, 'neu': 0.22, 'pos': 0.78, 'compound': 0.7964}

# Análise de texto com negação
s.polarity_scores('Eu não estou feliz')
#{'neg': 0.265, 'neu': 0.241, 'pos': 0.494, 'compound': 0.4404}

{'neg': 0.265, 'neu': 0.241, 'pos': 0.494, 'compound': 0.4404}